In [ ]:
%%bash
jupyter nbconvert models.ipynb --to script

In [ ]:
import torch
from torch import nn
from torch.autograd import Variable

import losswise

from prettytable import PrettyTable
from tqdm import tqdm
import numpy as np
import os
import sys
import random

from datasets import BurstDataset, ShuffledBatchSequentialSampler, FakeBurstDataset
from prep_dataset import BurstDatasetStandardizer
from models import Encoder, Decoder
from train_functions import *
from eval_functions import plot_autoencoding, autoencode


In [ ]:
# todo: implement cuda stuff

### Create fake dataset

In [ ]:
torch.manual_seed(1)
np.random.seed(1)
random.seed(1)

In [ ]:
amp = 1
phase_shift = None
freq = None
noise_std = 0.01
PAD_LENGTH = 500
dataset_len = 1000
test_dataset_len = 50
train_dataset = FakeBurstDataset(amp=amp, phase_shift=phase_shift, freq=freq, noise_std=noise_std, dataset_len=dataset_len, pad_len=PAD_LENGTH)
dev_dataset = FakeBurstDataset(amp=amp, phase_shift=phase_shift, freq=freq, noise_std=noise_std, dataset_len=test_dataset_len, pad_len=PAD_LENGTH)
test_dataset = FakeBurstDataset(amp=amp, phase_shift=phase_shift, freq=freq, noise_std=noise_std, dataset_len=test_dataset_len, pad_len=PAD_LENGTH)

In [ ]:
len(train_dataset)

### Initialize model

In [ ]:
HIDDEN_SIZE = 100
INPUT_SIZE = 1 # This CANNOT be changed! 
BIDIRECTIONAL = True
NUM_LAYERS = 1
EXTRA_INPUT_DIM = False
encoder = Encoder(INPUT_SIZE, HIDDEN_SIZE, bidirectional=BIDIRECTIONAL, num_layers=NUM_LAYERS)
decoder = Decoder(HIDDEN_SIZE, INPUT_SIZE, extra_input_dim=EXTRA_INPUT_DIM, encoder_bidirectional=BIDIRECTIONAL, 
                  num_layers=NUM_LAYERS)

### Define training params

In [ ]:
BATCH_SIZE = 30 
NUM_EPOCHS = 60 # Normally use 50, but can stop early at 20
LR = 1e-3
WEIGHT_DECAY = 0
# rate at which the probability of teacher_forcing decreases. 
TEACHER_FORCING_SLOPE = 0.01   # set this to None to turn off teacher forcing.
TRAIN_REVERSED = True

SAVE_DIR = None

BATCH_BY_LEN = True

# Run training

#### params_dict for fake dataset

In [ ]:
USE_LOSSWISE = False

In [ ]:
params_dict = {'pad length': PAD_LENGTH, 'noise_std': noise_std,  
               'amp': amp, 'freq':freq, 'phase shift': phase_shift,
         'train dataset len':len(train_dataset), 'test dataset len':len(test_dataset), 
               'hidden size': HIDDEN_SIZE, 'batch size':BATCH_SIZE, 
               'bidirectional':BIDIRECTIONAL, 'num layers':NUM_LAYERS, 
               'extra input dim':EXTRA_INPUT_DIM,
         'num epochs': NUM_EPOCHS, 'learning rate': LR, 'weight decay': WEIGHT_DECAY, 
          'teacher forcing slope': TEACHER_FORCING_SLOPE, 'train reversed':TRAIN_REVERSED, 'save dir': SAVE_DIR}

In [ ]:
if USE_LOSSWISE:
    losswise.set_api_key('W2TAMB3SZ') # api_key for "coma-eeg"
    session = losswise.Session(tag='Testing autoencoder with fake data', max_iter=NUM_EPOCHS,
                               params=params_dict)
    losswise_graph = session.graph('loss', kind='min')
else:
    losswise_graph = None

In [ ]:
train_model(train_dataset, dev_dataset, test_dataset, encoder, decoder, SAVE_DIR,
            num_epochs=NUM_EPOCHS, 
            batch_size=BATCH_SIZE, lr=LR, weight_decay=WEIGHT_DECAY, 
            teacher_forcing_slope=TEACHER_FORCING_SLOPE, train_reversed=TRAIN_REVERSED, batch_by_len=BATCH_BY_LEN, 
            losswise_graph=losswise_graph, params_dict=params_dict)
if USE_LOSSWISE:
    session.done()

## Plot the autoencoding

In [ ]:
sample = test_dataset[38]
mse = plot_autoencoding(sample, encoder, decoder, toss_encoder_output=False, reverse=TRAIN_REVERSED)

In [ ]:
works = {'amp': 1,
 'batch size': 20,
 'freq': 1,
 'hidden size': 100,
 'learning rate': 0.001,
 'noise_std': 0.01,
 'num epochs': 20,
 'pad length': 300,
 'phase shift': None,
 'save dir': None,
 'teacher forcing': True,
 'test dataset len': 50,
 'train dataset len': 200,
 'weight decay': 0}

works = {'amp': 1,
 'batch size': 10,
 'bidirectional': True,
 'extra input dim': False,
 'freq': None,
 'hidden size': 100,
 'learning rate': 0.001,
 'noise_std': 0.01,
 'num epochs': 40,
 'num layers': 1,
 'pad length': 300,
 'phase shift': None,
 'save dir': None,
 'teacher forcing slope': 0.15,
 'test dataset len': 50,
 'train dataset len': 600,
 'train reversed': True,
 'weight decay': 0}